# Customizing Run Names
[![Open In Collab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langsmith-cookbook/blob/main/tracing-examples/runnable-naming/run-naming.ipynb)

Every LangSmith run receives a name. This name is visible in the UI and can be employed later for querying a particular run. In the context of tracing chains constructed with LangChain, the default run name is derived from the class name of the invoked object.

For runs categorized as "Chain", the name can be configured by calling the [runnable](https://python.langchain.com/docs/expression_language/) object's `with_config({"run_name": "My Run Name"})` method. This guide illustrates its application through several examples.

**Note:** Only chains and general runnables support custom naming; LLMs, chat models, prompts, and retrievers do not.

Begin by installing the latest version of LangChain.


In [1]:
# %pip install -U langchain --quiet

In [1]:
import os

# Update with your API URL if using a hosted instance of Langsmith.
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "YOUR API KEY"  # Update with your API key
os.environ["LANGCHAIN_TRACING_V2"] = "true"
project_name = "YOUR PROJECT NAME"  # Update with your project name
os.environ["LANGCHAIN_PROJECT"] = project_name  # Optional: "default" is used if not set

In [2]:
from langsmith import Client

client = Client()

## Example 1: Simple Chain

In [3]:
from langchain import chat_models, prompts, callbacks, schema

chain = (
    prompts.ChatPromptTemplate.from_template("Reverse the following string: {text}")
    | chat_models.ChatOpenAI()
).with_config({"run_name": "StringReverse"})


with callbacks.collect_runs() as cb:
    for chunk in chain.stream({"text": "🌑🌒🌓🌔🌕"}):
        print(chunk.content, flush=True, end="")
    run = cb.traced_runs[0]

/Users/mukilloganathan/langchain/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


🌕🌔🌓🌒🌑

This will result in a trace that looks something like the following:

[![StringReverse](./img/moon_reverse.png)](https://smith.langchain.com/public/6fffe5a8-2561-4a73-bb7a-e40bc966ba33/r)

If you inspect the run object, you can see the run name is now "StringReverse". You can query within a project for runs with this name to see all the times this chain was used. Do so using the filter syntax `eq(name, "MyRunName")`.

In [4]:
print(f"Saved name {run.name}")

Saved name StringReverse


In [5]:
# List with the name filter to get runs with the assigned name
next(client.list_runs(project_name=project_name, filter='eq(name, "StringReverse")'))

Run(id=UUID('1960c94c-f860-4403-a1bd-3ebdd29099e0'), name='StringReverse', start_time=datetime.datetime(2024, 2, 13, 2, 33, 19, 580185), run_type='chain', end_time=datetime.datetime(2024, 2, 13, 2, 33, 20, 427903), extra={'runtime': {'sdk': 'langsmith-py', 'sdk_version': '0.0.87', 'library': 'langchain-core', 'platform': 'macOS-14.0-arm64-arm-64bit', 'runtime': 'python', 'py_implementation': 'CPython', 'runtime_version': '3.11.5', 'langchain_version': '0.1.6', 'thread_count': 15.0, 'mem': {'rss': 71892992.0}, 'cpu': {'time': {'sys': 2.74464128, 'user': 1.438423424}, 'ctx_switches': {'voluntary': 12209.0, 'involuntary': 0.0}, 'percent': 0.0}, 'library_version': '0.1.22'}, 'metadata': {}}, error=None, serialized={'id': ['langchain', 'schema', 'runnable', 'RunnableSequence'], 'kwargs': {'first': {'id': ['langchain', 'prompts', 'chat', 'ChatPromptTemplate'], 'kwargs': {'input_variables': ['text'], 'messages': [{'id': ['langchain', 'prompts', 'chat', 'HumanMessagePromptTemplate'], 'kwargs':

## Example 2: Runnable Lambda

LangChain's [RunnableLambdas](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.base.RunnableLambda.html) are custom functions that can be invoked, batched, streamed, and/or transformed.

By default (in langchain versions >= 0.0.283), the name of the lambda is the function name. You can customize this by calling `with_config({"run_name": "My Run Name"})` on the runnable lambda object.

In [6]:
from langchain_core.output_parsers import StrOutputParser


def reverse_and_concat(txt: str) -> str:
    return txt[::-1] + txt


lambda_chain = chain | StrOutputParser() | reverse_and_concat

In [7]:
with callbacks.collect_runs() as cb:
    print(lambda_chain.invoke({"text": "🌑🌒🌓🌔🌕"}))
    # We will fetch just the lambda run (which is the last child run in this root trace)
    run = cb.traced_runs[0].child_runs[-1]

🌑🌒🌓🌔🌕🌕🌔🌓🌒🌑


In [8]:
from langchain.callbacks.tracers import langchain

# If you are using LangChain < 0.0.283, this will be "RunnableLambda"
langchain.wait_for_all_tracers()
print(f"Saved name: {run.name}")

Saved name: reverse_and_concat


The lambda function's trace will be given the lambda function's name, `reverse_and_concat`, as shown below:

<a href="https://smith.langchain.com/public/86ff7b17-4a84-4f99-8a98-13c49658af89/r/54ea562c-d30a-4d71-8c98-5c174819c9dd" target="_blank"><img src="img/reverse_and_concat.png" alt="reverse_and_concat" width="75%"></a>

#### Customize Lambda Name

In the `lambda_chain` above, our function was automatically promoted to a "RunnableLambda" via the piping syntax.
We can customize the run name using the `with_config` syntax once the object is created.

In [9]:
from langchain.schema import runnable

configured_lambda_chain = (
    chain
    | StrOutputParser()
    | runnable.RunnableLambda(reverse_and_concat).with_config(
        {"run_name": "LambdaReverse"}
    )
)

In [10]:
with callbacks.collect_runs() as cb:
    print(configured_lambda_chain.invoke({"text": "🌑🌒🌓🌔🌕"}))
    run = cb.traced_runs[0].child_runs[-1]

🌑🌒🌓🌔🌕🌕🌔🌓🌒🌑


In [11]:
callbacks.tracers.langchain.wait_for_all_tracers()
print(f"Saved name: {run.name}")

Saved name: LambdaReverse


[![LambdaReverse](img/custom_lambda.png)](https://smith.langchain.com/public/43eea194-b0f0-452b-b92c-4ae39ffd0799/r)

The lambda function's name now will be `LambdaReverse`, as shown below:


## Example 3: Agents

Since LangChain agents and agent executors are types of chains.

In [12]:
from langchain import agents, tools


agent_executor = agents.initialize_agent(
    llm=chat_models.ChatOpenAI(),
    tools=[tools.ReadFileTool(), tools.WriteFileTool(), tools.ListDirectoryTool()],
    agent=agents.AgentType.OPENAI_FUNCTIONS,
)

/Users/mukilloganathan/langchain/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [13]:
with callbacks.collect_runs() as cb:
    result = agent_executor.with_config({"run_name": "File Agent"}).invoke(
        "What files are in the current directory?"
    )
    run = cb.traced_runs[0]
    print(result["output"])

The files in the current directory are:
1. run-naming.ipynb
2. img
3. .ipynb_checkpoints


In [14]:
callbacks.tracers.langchain.wait_for_all_tracers()
print(f"Saved name: {run.name}")

Saved name: File Agent


The resulting agent trace will reflect the custom name you've assigned to it.

[![File Agent Trace](img/file_agent.png)](https://smith.langchain.com/public/00537050-0da5-4f95-ba28-857183ae9b0c/r)

## Conclusion

An easy way to customize run names is to use the `with_config` syntax on your LangChain chain or runnable lambda. This makes it easier to understand a trace at a glance.